# Imports

In [10]:
import pandas as pd
import geopandas as gpd
import numpy as np
from functools import reduce

# Load data

In [11]:
# LSOA boundaries
raw_lsoa = gpd.read_file("datasets/1_original/lsoa/LSOA_2021_EW_BSC_V4.shp")

# Self-reported health by LSOA
raw_health = pd.read_excel("datasets/1_original/health/self_reported_health.xlsx")

# Age and sex of residents by LSOA
raw_age_sex = pd.read_excel("datasets/1_original/moderating/age_sex.xlsx")

# Ethnicity of residents by LSOA
raw_ethnicity = pd.read_excel("datasets/1_original/moderating/ethnicity.xlsx")

# Index of Multiple Deprivation by LSOA
raw_deprivation = pd.read_excel("datasets/1_original/moderating/deprivation.xlsx")

# Ordinance Survey OpenGreenspace
raw_os_greenspace = gpd.read_file("datasets/1_original/greenspace/os_greenspace/data/TQ_GreenspaceSite.shp")

# Urban Atlas greenspace
raw_ua_greenspace = gpd.read_file("datasets/1_original/greenspace/ua_greenspace/ua_greenspace_london.gpkg")


In [12]:
lsoa = raw_lsoa.copy()
health = raw_health.copy()
age_sex = raw_age_sex.copy()
ethnicity = raw_ethnicity.copy()
deprivation = raw_deprivation.copy()
os_greenspace = raw_os_greenspace.copy()
ua_greenspace = raw_ua_greenspace.copy()
dfs = [lsoa, health, age_sex, ethnicity, deprivation, os_greenspace, ua_greenspace]

# Clean column names

In [13]:
# Strip whitespace and replace spaces with underscores
for df in dfs:
    df.columns = df.columns.str.strip().str.replace(' ', '_').str.lower()

# Standardise names
lsoa = lsoa.rename(columns={'lsoa21cd': 'lsoa'})
health = health.rename(columns={'lsoa_code': 'lsoa'})
age_sex = age_sex.rename(columns={'lsoa_2021_code': 'lsoa'})
ethnicity = ethnicity.rename(columns={'lsoa_code': 'lsoa', 'other_any_other': 'any_other'})
deprivation = deprivation.rename(columns={'lsoa_code_(2011)': 'lsoa', 'index_of_multiple_deprivation_(imd)_rank_(where_1_is_most_deprived)':'imd'})


# Drop unneeded columns

In [14]:
lsoa = lsoa[['lsoa', 'geometry']]
health = health.drop(columns = ['local_authority_code', 'local_authority_name', 'good_health', 'fair_health', 'bad_health', 'very_bad_health'])
os_greenspace = os_greenspace[['geometry']]
age_sex = age_sex.drop(columns = ['lad_2021_code', 'lad_2021_name', 'lsoa_2021_name'])
ethnicity = ethnicity.drop(columns=['local_authority_name', 'local_authority_code'])
deprivation = deprivation[['lsoa', 'imd']]

# Align coordinate systems

In [15]:
lsoa = lsoa.to_crs('EPSG:3035')
os_greenspace = os_greenspace.to_crs('EPSG:3035')

# Convert health figure to proportion

In [16]:
health['very_good_health'] = health["very_good_health"] / health["all_usual_residents"].replace(0, np.nan)
health = health.drop(columns=['all_usual_residents'])

# Combine LSOA and demographic datasets

In [17]:
demographic_datasets = [health, age_sex, ethnicity, deprivation]

# Inner merge drops LSOAs without data from all dfs (i.e. those outside London)
merged_demographic_datasets = reduce(lambda left, right: left.merge(right, on='lsoa', how='inner'), demographic_datasets)

# Inner merge drops LSOAs outside London
lsoa = lsoa.merge(merged_demographic_datasets, on='lsoa', how='inner')

# Save output

In [18]:
lsoa.to_file("datasets/2_cleaned/cleaned_lsoa.gpkg", driver="GPKG", index = False)
os_greenspace.to_file("datasets/2_cleaned/cleaned_os_greenspace.gpkg", driver="GPKG", index = False)
ua_greenspace.to_file("datasets/2_cleaned/cleaned_ua_greenspace.gpkg", driver="GPKG", index = False)